# Predict bad LS moves

This notebook develops simple models for predicting bad local search moves. Particularly, given nodes $U$ and $V$ in routes $R_U$ and $R_V$, it predicts whether each LS operator we currently have is likely to produce an improving solution if the operator were applied to these node pairs $U$ and $V$.

The motivation is that evaluating a full operator move is typically somewhat slow, whereas a fast and reasonably accurate prediction method can completely avoid such evaluations.

In [1]:
%cd ..

D:\Projects\Python\Euro-NeurIPS-2022


In [12]:
%matplotlib inline

In [10]:
from collections import defaultdict
from dataclasses import dataclass
from glob import iglob
import re

import numpy as np
import matplotlib.pyplot as plt

## Parsing utilities

These can be used to quickly parse the raw results for a single instance into something that contains the same data, but in a more workable format. Uses a generator, so the memory overhead is minimal.

In [7]:
@dataclass
class Route:
    idx: int
    clients: list[int]

@dataclass
class Record:
    op: int
    U: int
    V: int
    delta: int
    Ru: Route
    Rv: Route    

def parse_file(file: str):
    def parse_record(record: list[str]) -> Record:
        op = int(record[0].strip())
        U, V, delta = map(int, record[1].strip().split(" "))

        idx, *clients = map(int, re.findall('[0-9]+', record[2].strip()))
        Ru = Route(idx, clients)

        idx, *clients = map(int, re.findall('[0-9]+', record[3].strip()))
        Rv = Route(idx, clients)

        return Record(op, U, V, delta, Ru, Rv)

    with open(file, 'r') as fh:
        args = [iter(fh)] * 4
        records = zip(*args)

        yield from map(parse_record, records)

In [15]:
for record in parse_file('data/raw/ORTEC-VRPTW-ASYM-1de83915-d1-n262-k15.txt'):
    pass